# Projet maths-informatique octobre 2019: Jean-Baptiste Flieller et Arthur Wienhold

In [ ]:
import autograd                       #prérequis au niveau de l'environement
from autograd import numpy as np
import matplotlib.pyplot as plt
from math import *


L'objectif de ce projet était le tracé informatique de lignes de niveaux.
Plus précisement, l'utilisateur saisit  f une fonction de deux variables à valeurs réelles, continuement différenciable, et le programme doit pouvoir représenter sur un graphique l'ensemble {(x,y), f(x,y)=c} avec c un réel donné en argument.
 Nous chercherons par plusieurs approches à remplir cette mission, avec deux contraintes principales:
 - Le temps de calcul qui ne doit pas excéder une durée d'attente raisonnable, compte tenu cependant de la complexité de la fonction à tracer, nous y renviendrons
 - La précision de la courbe rendue, qui là encore est soumis à la contrainte ci-dessus, ainsi qu'à la fonction.
 

Deux approches ont été envisagées, l'une naïve et l'autre plus élaborée, mais toutes deux reposent sur une méthode commune: le pavage de l'espace: on subdivise le plan en carrés de tailles égales, dont le côté sera un argument donné par l'utilisateur stocké dans la variable taille_cell, valant par défaut . Un carré sera par ailleurs repéré dans l'espace par les coordonnées de son coin inférieur gauche.


# Une première approche naïve: une approche par dichotomie uniquement.

Le principe global de cette méthode est le suivant: nous subdivisons chaque carré en 10 "tranches" de largeur (abscisses) 0.1taille_cell et de hauteur taille_cell.
Puis, sur chacune des colonnes, nous appliquons un algorithme de dichotomie à la fonction g qui, pour une abscisse abs_x fixée, donne à y f(abs_x,y)-c. Nous trouvons alors un point de la ligne de niveau, si il existe, sur chacune des colonnes, puis on relie tous ces points, "dans bon ordre", à la fin.


In [ ]:
def find_seed_vnaive(f, c=0.5, x=0, eps=2**(-26)):
    def g(x,y):     # nous voulons résoudre f(x,y)-c=0, nous créons donc la fonction g=f-c
        return f(x,y)-c
    if g(x,0)*g(x,1)>0:   #nous testons s'il existe raisonnablement un tel y par théorème des valeurs intermédiaires
        return None
    ymin=0
    ymax=1
    while abs(g(x,ymin)-g(x,ymax))>eps:  #nous effectuons une dichotomie sur une colonne à x fixé
        ymil=(ymin+ymax)/2               #nous stoppons l'algo avec la précision souhaitée
        if g(x,ymil)*g(x,ymin)>0:
            ymin=ymil
        else:
            ymax=ymil
    return (ymin+ymax)/2


On notera que le succès de l'algorithme de dichotomie, qui nécessite la présence d'une fonction monotone et assez régulière sur la colonne avec un zéro unique, repose sur un pavage assez fin du plan pour que localement les lignes de niveaux puissent être approximées par des segments. Cependant, plus le pavage est fin, plus on doit faire de calculs, il faut donc trouver un compromis lors de l'utilisation du programme. 

In [ ]:
def simple_contour(f, c=0.5, delta=0.01): # dans sa version naive, il suffit d'effectuer un find_seed en changeant l'abscisse x
    X=[0]                                 # en le déplaçant de delta à  chaque itération
    Y=[find_seed_vnaive(f, c=c)]
    if Y[0]==None:
        return [],[]
    
    n=int(1//delta)
    for k in range(1,n+2):            #nous effectuons n+1 itérations
        A=find_seed_vnaive(f, c=c, x=X[-1])
        if not A==None:
            X.append(X[-1]+delta)
            Y.append(A)      
    return np.array(X), np.array(Y)

In [ ]:
def f(x,y):
    return 2*(np.exp(-x**2-y**2)-np.exp(-(x-1)**2-(y-1)**2))   #la fonction que nous utilisons pour les test

X,Y=simple_contour(f)  #nous effectuons un test sur un morceau de la fonction restreint [0,1]**2

plt.plot(X,Y)
plt.show()

Puis, on généralise le tracé à tout le plan en faisant des tracés dans chaque cellule, en faisant "tourner" les cellules de façon à conserver une fonction contour simple dont la graine, si elle existe, est dans sur le côté gauche du carré.

In [ ]:
LEFT, UP, RIGHT, DOWN = 0, 1, 2, 3  # clockwise


def rotate_direction(direction, n=1):
    return (direction + n) % 4


def rotate(x, y, n=1):
    if n == 0:
        return x, y
    elif n >= 1:
        return rotate(1 - y, x, n - 1)
    else:
        assert n < 0
        return rotate(x, y, n=-3 * n)


def rotate_function(f, n=1):
    def rotated_function(x, y):
        xr, yr = rotate(x, y, -n)
        return f(xr, yr)

    return rotated_function


# Complex Contouring
# ------------------------------------------------------------------------------

# Customize the simple_contour function used in contour :
# simple_contour = smart_simple_contour


def contour(f, c, xs=[0.0, 1.0], ys=[0.0, 1.0], delta=0.01):
    curves = []
    nx, ny = len(xs), len(ys)
    for i in range(nx - 1):
        for j in range(ny - 1):
            xmin, xmax = xs[i], xs[i + 1]
            ymin, ymax = ys[j], ys[j + 1]

            def f_cell(x, y):
                return f(xmin + (xmax - xmin) * x, ymin + (ymax - ymin) * y)

            done = set()
            for n in [0, 1, 2, 3]:
                if n not in done:
                    rotated_f_cell = rotate_function(f_cell, n)
                    x_curve_r, y_curve_r = simple_contour(rotated_f_cell, c, delta)
                   
                    
                    exit = None
                    if len(x_curve_r) >= 1:
                        xf, yf = x_curve_r[-1], y_curve_r[-1]
                        if xf == 0.0:
                            exit = LEFT
                        elif xf == 1.0:
                            exit = RIGHT
                        elif yf == 0.0:
                            exit = DOWN
                        elif yf == 1.0:
                            exit = UP
                    if exit is not None:  # a fully successful contour fragment
                        exit = rotate_direction(exit, n)
                        done.add(exit)
                    

                    x_curve, y_curve = [], []
                    for x_r, y_r in zip(x_curve_r, y_curve_r):
                        x, y = rotate(x_r, y_r, n=-n)
                        x_curve.append(x)
                        y_curve.append(y)
                   
                    x_curve = np.array(x_curve)
                    y_curve = np.array(y_curve)
                    
                    curves.append((xmin + (xmax - xmin) * x_curve, ymin + (ymax - ymin) * y_curve))
    return curves
        


In [ ]:
def decoupe(a, b, d): # nous écrivons une fonction qui forme des listes d'entiers allant de a à  b et espace de d
    L=[]
    n=int((b-a)//d)
    for k in range(n+2):
        L.append(a+k*d)
    return L

Xs=decoupe(-2, 3, 0.2)
Ys=decoupe(-1, 2, 0.2)
for i in range(7):
    level_curves = contour(f, c= -1.5+i/2, xs=Xs, ys=Ys)
    for x, y in level_curves:   # nous affichons l'ensemble des courbes obtenues avec la version naïve
        plt.plot(x, y, 'm')
#le calcul va prendre 2-3 minutes...

On constate donc que cette méthode, bien que rudimentaire, propose des résultats satisfaisants, mais avec une quantité gigantesque de calculs, et une approche assez rudimentaire. Cherchons donc une solution plus avancée, aussi bien sur le plan mathématique et informatique

#                                         Méthode de Newton 


Principe: on pave l'espace, et on va comme dans la première méthode trouver un point de la courbe de niveau sur le côté gauche du carré.
Puis, on cherche à tracer le segment de courbe de niveau, si il existe, contenu dans le carré, les points étant séparé d'environ delta, avec delta=01taille_cell
Le principe est le suivant: on définit la fonction F qui à (x,y) donne (f(x,y)-c, distance((x,y),(w,z))^2-delta^2) avec (w,z) un point de la ligne de niveau déjà trouvé, en argument de la fonction F.
Le but est de trouver un zéro à cette fonction: un point de la ligne de niveau à une distance delta du précédent.

## Explication mathématique 

Nous définissons Fx et Fy les composantes en x et y de F
 
Un developpement limité au premier ordre donne, avec (x0,y0) un point du plan: 

$ F_x(x,y) = F_x(x_0,y_0)+ \frac{\partial F_x}{\partial x}(x_0,y_0)(x-x_0) + \frac{\partial F_x}{\partial y}(x_0,y_0)(y-y_0) + o(x-x_0,y-y_0)$

$ F_y(x,y) = F_y(x_0,y_0)+ \frac{\partial F_y}{\partial x}(x_0,y_0)(x-x_0) + \frac{\partial F_y}{\partial y}(x_0,y_0)(y-y_0) + o(x-x_0,y-y_0)$

Nous conserverons cette approximation.

Si on veut converger vers un point (x,y) qui annule F, nous tirons des deux lignes précédentes une relation: 

$\begin{pmatrix}
- F_x(x_0,y_0) \\
- F_y(x_0,y_0)
\end{pmatrix}
= J(x_0,y_0)
\begin{pmatrix}
x_1 - x_0\\
y_1-y_0
\end{pmatrix}
$

Puis par récurrence, nous obtenons la formule de l'algorithme de Newton généralisée en dimension 2:

$\begin{pmatrix}
x_{n+1}\\
y_{n+1}
\end{pmatrix}
=
\begin{pmatrix}
x_{n}\\
y_{n}
\end{pmatrix}
- J^{-1}(x_n,y_n)
\begin{pmatrix}
F_x(x_n,y_n)\\
F_y(x_n,y_n)
\end{pmatrix}
$



## Partie informatique

Préambule:
  - Dans un premier temps, le lecteur peut ignorer l'argument "side" dans les fonctions qui ne sera utilisé que pour le contour_complexe. Qu'il le considère égal à 1 (ce qui sera sa valeur par défaut dans contour simple) pour l'instant.
  - La partie contour complexe de cette méthode n'aboutit pas malgré un temps très conséquent passé dessus, nous y reviendrons après contour_simple. Cependant, nous avons laissé en place tous les outils qui devaient la faire fonctionner, d'où les arguments side.

Notre méthode va être la suivante: on trouve une seed sur le côté gauche du carré, puis pour trouver un nouveaux point, on "saute" dans la direction de la ligne de niveau, qui est orthogonale au gradient, puis on corrige l'erreur avec l'algorithme de Newton

In [ ]:
def find_seed_side(f, c=0.5, abs_x=0, ord_y=0,taille_cell=(1,1), eps=2**(-26)):
    def g(x,y):     
        return f(x,y)-c
    if g(abs_x,ord_y)*g(abs_x,ord_y+taille_cell[1])>0:   
        return None
    ymin=ord_y
    ymax=ord_y+1
    while abs(g(abs_x,ymin)-g(abs_x,ymax))>eps:  
        ymil=(ymin+ymax)/2               
        if g(abs_x,ymil)*g(abs_x,ymin)>0:
            ymin=ymil
        else:
            ymax=ymil
    return (ymin+ymax)/2
    
#inutiles dans la version contour_simples, ces deux fonctions permettent de trouver une seed sur n'importe quel côté du carré
def find_seed_x(f, c=0.5, abs_x=0, ord_y=0,taille_cell=(1,1), eps=2**(-26)):
    def g(x,y):     
        return f(x,y)-c
    if g(abs_x,ord_y)*g(abs_x+taille_cell[0],ord_y)>0:   
        return None
    xmin=abs_x
    xmax=abs_x+1
    while abs(g(xmin,ord_y)-g(xmax,ord_y))>eps:  
        xmil=(xmin+xmax)/2               
        if g(xmil,ord_y)*g(xmin,ord_y)>0:
            xmin=xmil
        else:
            xmax=xmil
    return (xmin+xmax)/2





def find_seed_side(f,c,abs_x,ord_y,taille_cell,eps,side=1):
    if side==1:
        return find_seed(f,c,abs_x,ord_y,taille_cell,eps)
    elif side==2:
        return find_seed_x(f,c,abs_x,ord_y,taille_cell,eps)  #le côté gauche est side=1, puis on tourne dans le sens trigo
    elif side==3:
        return find_seed(f,c,abs_x+taille_cell[0],ord_y,taille_cell,eps)
    else:
        return find_seed_x(f,c,abs_x,ord_y+taille_cell[1],taille_cell,eps)
    

In [ ]:
#donne le gradient de f au point (x,y)
def grad(f,x,y):
    g=autograd.grad
    return np.array([g(f,0)(x,y),g(f,1)(x,y)])


#On construit la jacobienne de F en un point (x,y)

    
def Jacobienne_F(f,x,y):
    j=autograd.jacobian
    return np.c_[j(f,0)(x,y),j(f,1)(x,y)]


def prod_scalaire(a,b):
    return a[0]*b[1]+a[1]*b[0]

#retourne un vecteur, de norme delta, orthogonal au gradient de f en (x,y), donc tangent à la ligne de niveau de f en (x,y)
def tang_norm(f,x,y,delta=0.01):
    g=autograd.grad
    normed=delta/((g(f,0)(x,y)**2+g(f,1)(x,y)**2)**(0.5))
    return np.r_[g(f,1)(x,y)*normed,-g(f,0)(x,y)*normed]



    
#Trouve le point suivant, partant de (xi,yi)  

def trouve_suivant(f,xi,yi,d,c,eps):

    def F(x,y):
        return np.array([f(x,y)-c,(x-xi)**2+(y-yi)**2-d**2])

    (x,y)=(xi,yi)+tang_norm(f,xi,yi,d)

    compteur=0      #compte le nombre d'itérations, ce qui rajoute une sécurité sur l'arrêt de la boucle

    while F(x,y)[0]**2+F(x,y)[1]**2 > eps**2 and compteur<100000:
        J=Jacobienne_F(F,x,y)
        try:
            (x,y)=(x,y)-np.dot(np.linalg.inv(J),F(x,y))
        except np.linalg.LinAlgError :
            return (x,y)
        compteur+=1

    return (x,y)


Nous sommes maintenant prêts à mettre en place le programme qui tracera le segment de ligne dans le carré. Notons que ce programme s'arretera quand on sera proche des côtés haut, droite et bas de la cellule : on part de sur le côté gauche, et l'on suppose que le découpage du plan est suffisament fin pourque la ligne de niveau de f ne coupe qu'une seule fois ce côté.

Notons aussi qu'au départ du côté gauche du carré, il faut faire le saut vers l'intérieur de la cellule, d'où ces petits programmes spécifiques à la première recherche (encore une fois, ignorons pour l'instant le paramètre side qui vaut 1 par défaut dans simple_contour, on regarde donc le côté gauche).

In [ ]:
def normalise_dim2(vecteur):
    norme=sqrt(vecteur[1]**2+vecteur[0]**2) 
    return (1/norme)*vecteur

def choose_direction_init_side(f,x,y,side):
    direction=normalise_dim2(np.array([-grad(f,x,y)[1],grad(f,x,y)[0]]))
    if side==1:
        if direction[0]>=0:
            return direction
        else:
            return -direction
    if side==3:
        if direction[0]<=0:
            return direction
        else:
            return -direction
    if side==2:
        if direction[1]>=0:
            return direction
        else:
            return -direction
    else:
        if direction[1]<=0:
            return direction
        else:
            return -direction

    
def trouve_suivant_init_side(f,xi,yi,d,c,side,eps):

    def F(x,y):
        return np.array([f(x,y)-c,(x-xi)**2+(y-yi)**2-d**2])

    (x,y)=(xi,yi)+choose_direction_init_side(f,xi,yi,side)*d       #le seul changement est qu'on utilise ce "saut"


    compteur=0 

    while F(x,y)[0]**2+F(x,y)[1]**2 > eps**2 and compteur<1000:
        J=Jacobienne_F(F,x,y)
        try:
            (x,y)=(x,y)-np.dot(np.linalg.inv(J),F(x,y))
        except np.linalg.LinAlgError :
            return (x,y)
        compteur+=14
    return (x,y)

In [ ]:
def simple_contour_side(f,abs_x,ord_y,c=0.0,side=1,taille_cell=(1,1),eps=2**(-26)):
    d=0.01*taille_cell[0]
    X,Y=np.array([]),np.array([])
    if side==1 or side==3:
        start_y=find_seed_side(f,c,abs_x,ord_y,taille_cell,eps,side)
        start_x=abs_x

    else:
        start_x=find_seed_side(f,c,abs_x,ord_y,taille_cell,eps,side)
        start_y=ord_y


    if isinstance(start_y,float) and isinstance(start_x,float):
        X=np.append(X,start_x)
        Y=np.append(Y,start_y)
        (xi,yi)=trouve_suivant_init_side(f,start_x,start_y,d,c,side,eps)
        X=np.append(X,xi)
        Y=np.append(Y,yi)


        compteur=0
        while xi>eps+abs_x and (abs_x+taille_cell[0]-eps)>xi and yi>eps+ord_y and (ord_y+taille_cell[1]-eps)>yi and compteur <1000:
            (xi,yi)=trouve_suivant(f,xi,yi,d,c,eps)
            X=np.append(X,xi)
            Y=np.append(Y,yi)
            compteur+=1
              #plt.plot(X,Y)
              #plt.show()
        return (X,Y)
    
    else:
        return (X,X)

In [ ]:
def cercle(x,y):
    return x**2+y**2

simple_contour_side(cercle,-0.3,0.,0.5,1)


### Partie contour complexe

Le principe consistait de manière simple à, pour chaque cellule, tracer les simples_contour_side partant des 4 côtés, puis à concaténer tous les segmments de courbe. Pour optimiser le processus, il s'agit de mettre en place certains garde fous: par exemple, si à partir de la gauche, on fait un segment de courbe qui sort par la droite, il est inutile de faire le tracé qui part de la droite et qui sera inutile ! Cette dernière barrière n'a pas été implémentée par manque de temps.
Le programme contour devient alors assez simple:

In [ ]:
def contour_newton(f,c=0.,xc=[0.,1.],yc=[0.,1.0],delta=0.01,eps=2**(-26)):
    xs,ys=[],[]
    for i,abs_x in enumerate(xc[:(len(xc)-1)]):
        for j,ord_y in enumerate(yc[:(len(yc)-1)]):
            taille_cell=(xc[i+1]-xc[i],yc[j+1]-yc[j])
            for side in range (1,5):
                xs+=[simple_contour_side(f,abs_x,ord_y,c,side,taille_cell,eps)[0]]
                ys+=[simple_contour_side(f,abs_x,ord_y,c,side,taille_cell,eps)[1]]

    for x,y in zip(xs,ys):
        plt.plot(x,y)
    plt.show()
contour(cercle,0.5,[-0.7,0.,0.7],[-0.7,0.,0.7])


Pourquoi ça ne marche pas ? Parceque pour une raison inconnue, le tracé simple_contour_side donne des résultats aberrants lorsqu'il ne "part pas du bon côté". 
Exemple: 

In [ ]:
simple_contour_side(cercle,-0.3,0.,0.5,1) #ok

In [ ]:
simple_contour_side(cercle,-0.3,0.,0.5,3) #mais...(rappelons que 3 correspond au côté droit du carré, pas de raison que ça ne marche pas à priori...)


Et malgré de nombreuses heures de bataille et l'introduction de fonctions plus élaborées, comme celle-ci qui choisit la "bonne direction" parmis les deux possible quand on prend l'orthogonal au gradient, celle-ci étant celle de plus grand angle par rapport au segment qui relie le point actuel et le point précédent (fonction continuement différenciable donc pas de point de rebroussement). Notre problème en l'occurence était que le sens de l'inégalité varie puisque le produit scalaire est proportionel au cosinus de l'angle, et même quand ça fonctionne bien, ça n'apporte presque rien au problème (sauf dans certains cas isolés).

In [ ]:
def prod_scalaire(a,b):
    return a[0]*b[1]+a[1]*b[0]
def choose_direction(f,x,y,pt_prec):
    direction1=normalise_dim2(np.array([-grad(f,x,y)[1],grad(f,x,y)[0]]))
    direction2=-direction1
    if prod_scalaire(direction1,np.array([x-pt_prec[0],y-pt_prec[1]]))>=prod_scalaire(direction2,np.array([x-pt_prec[0],y-pt_prec[1]])):
        return direction1
    else:
        return direction2  

Pour montrer cependant qu'on a pas tout perdu, on peut quand même "tricher" mais c'est purement empirique, en disant au programme de quels côtés il doit partir, pour avoir le graphique complet. 

# Conclusion

En conclusion, nous présentons une comparaison obtenue grâce à la biblothèque time: pour l'appel contour_simple(cercle,0.,0.,0.5) (soit le tracé d'un quart de cercle), la méthode "élaborée" met 0.0265s à s'exécuter contre 0.1s pour la méthode naïve: on peut donc présumer que sur de grands graphiques, la méthode faisant appel au calcul différentiel et à la vitesse de calcul du module numpy sera bien plus efficace, sous réserve de  lui apporter les ajustements qui lui permettront de fonctionner pleinement.  